In [1]:
%pip install langchain langchain_community langchain-chroma faiss-gpu pymupdf sentence-transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.5/612.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.1 MB/s eta 0:00:00

In [138]:
# Import necessary libraries from LangChain and other packages

from langchain.document_loaders import PyMuPDFLoader
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain , RetrievalQA
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Load a PDF document from a specified file path
def load_pdf_document(file_path):
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()
    return documents


pdf_file_path = "AI_and_Technology_Knowledge_Base.pdf"
documents = load_pdf_document(pdf_file_path)


# Initialize a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Size of each text chunk
    chunk_overlap=100,  # Overlap between chunks to maintain context
    separators=[  # List of separators to determine where to split the text
        "\n\n",
        ".",
        "\n",
        " ",
        "",
    ]
)

# Apply the text splitter to the loaded documents to create chunks of manageable size
docs = text_splitter.split_documents(documents)

# Function to remove whitespace and newlines from the content of each document
def remove_ws(d):
    text = d.page_content.replace('\n','')
    d.page_content = text
    return d

# Apply the whitespace removal function to all documents
docs = [remove_ws(d) for d in docs]


In [139]:
# Initialize the embedding model using the pre-trained 'all-MiniLM-L6-v2' model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# This model is optimized for producing high-quality sentence embeddings efficiently.

# Create embeddings for the documents using SentenceTransformerEmbeddings
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Use FAISS to create a vector store from the documents and their embeddings
vector_store = FAISS.from_documents(docs, embedding)

In [170]:
# Cache for frequently asked queries
cache = defaultdict(str)

def retrieve_with_confidence(query, k=3, threshold=0.6):
    """
    Retrieves documents based on the query with a confidence score.

    Args:
        query (str): The search query for which to retrieve documents.
        k (int): The number of top documents to retrieve (default is 3).
        threshold (float): The confidence score threshold for filtering documents (default is 0.6).

    Returns:
        tuple: A list of high confidence documents and a boolean indicating low confidence.
    """
    # Get query embedding
    query_embedding = embedding_model.encode([query])

    # Retrieve top-k documents using similarity search
    retrieved_docs = vector_store.similarity_search(query, k=k)

    # Calculate cosine similarity scores
    doc_embeddings = [embedding_model.encode(doc.page_content) for doc in retrieved_docs]
    scores = cosine_similarity(query_embedding, doc_embeddings)[0]

    # Filter documents based on threshold
    high_confidence_docs = [doc for doc, score in zip(retrieved_docs, scores) if score >= threshold]
    low_confidence = len(high_confidence_docs) == 0  # Check if none of the scores meet the threshold

    # Return high confidence documents and low confidence flag
    return high_confidence_docs, low_confidence

In this implementation, I used both the GPT-Neo and GPT-2 models to leverage their unique strengths: GPT-Neo, being a larger model, provides more comprehensive and contextually aware text generation, while GPT-2 offers a lighter, faster alternative for tasks where computational resources or response time are a concern.

In [106]:
# Specify the model name for GPT-Neo
model_name = "EleutherAI/gpt-neo-1.3B"

# Load the tokenizer for the GPT-Neo model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set padding token for models that lack one
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define the text generation pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.7,
    truncation=True,
    clean_up_tokenization_spaces=True
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [107]:
# Specify the model name for GPT 2
model_name = "gpt2"

# Load the tokenizer for the GPT-Neo model
tokenizer2 = AutoTokenizer.from_pretrained(model_name)
model2 = AutoModelForCausalLM.from_pretrained(model_name)

# Set padding token for models that lack one
if tokenizer2.pad_token is None:
    tokenizer2.pad_token = tokenizer2.eos_token

# Define the text generation pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model2,
    tokenizer=tokenizer2,
    max_new_tokens=200,
    temperature=0.7,
    truncation=True,
    clean_up_tokenization_spaces=True
)
llm2 = HuggingFacePipeline(pipeline=hf_pipeline)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [181]:
def generate_answer(query, model):
    """
    Generates an answer to the given query using a retrieval system and an LLM chain based on a language model.

    Args:
        query (str): The user's question for which an answer is to be generated.
        model: The language model used for generating the answer.

    Returns:
        str: The generated answer based on the context retrieved.
    """
    # Check if the query is already cached to avoid redundant processing
    if query in cache:
      return cache[query]

    # Retrieve documents and confidence level
    retrieved_docs, low_confidence = retrieve_with_confidence(query)

    if low_confidence:
        return "Could you please clarify your question?"

    # Combine the retrieved documents for context
    context = " ".join([doc.page_content for doc in retrieved_docs])

    # Define the prompt template
    template = """Answer the question based on the given context.

                  Context: {context}

                  Question: {question}

                  Answer: """

    # Create prompt and LLM chain for the specified model
    prompt = ChatPromptTemplate.from_template(template)

    llm_chain = LLMChain(prompt=prompt, llm=model)

    # Generate the final answer
    response = llm_chain.invoke({"context": context, "question": query})

    # Store the response in the cache
    cache[query] = response

    return response

In [123]:
# Example query
query = "Can you explain AI?"

# Generate an answer using the GPT-Neo model
answer_gpt_neo = generate_answer(query, llm)
answer_start = answer_gpt_neo['text'].find("Answer:") + len("Answer:")
answer = answer_gpt_neo['text'][answer_start:].strip()  # Extracting and stripping whitespace

# Print only the answer
print(answer)

AI is defined as a computer system that is able to do a task that would be impossible for a human being.The definition of artificial intelligence (AI) is very broad. AI can be defined as any machine that has the ability to perform a task that is impossible for a human being.


In [129]:
# Example query
query = "Can you explain AI?"

# Generate an answer using the GPT 2 model
answer_gpt2 = generate_answer(query, llm2)
answer_start = answer_gpt2['text'].find("Answer:") + len("Answer:")
answer = answer_gpt2['text'][answer_start:].strip()  # Extracting and stripping whitespace

# Print only the answer
print(answer)

AI is a combination of two things: human intelligence and machine intelligence. For example, a computer can be programmed todo any number of tasks. One of the most important AI tasks is to learn.


The following function, `generate_answer2`, provides an alternative approach to answer generation using the RetrievalQA framework.
This setup combines document retrieval with language model generation, retrieving relevant documents and incorporating
them into a prompt that guides the language model to generate a response based on the specified context.


In [172]:
def generate_answer2(query, model):
    """
    Generates an answer to the provided query using the RetrievalQA framework.

    Args:
        query (str): The user's question for which an answer is to be generated.
        model: The language model used for generating the answer.

    Returns:
        str: The generated answer based on the context retrieved.
    """

    # Define the prompt template to structure the input for the LLM
    prompt_template = """
        Context: {context}
        Question: {question}
        Answer:   """
    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # Set up the RetrievalQA chain, specifying the language model, chain type, and retriever
    retrievalQA = RetrievalQA.from_chain_type(
        llm=model,  # The language model to generate answers
        chain_type="stuff",  # This indicates that all retrieved documents will be used to formulate the answer
        retriever=vector_store.as_retriever(search_kwargs={'k': 3}),  # Retrieves the top 3 relevant documents
        return_source_documents=True,  # Optionally return the source documents along with the answer
        chain_type_kwargs={"prompt": PROMPT}  # Use the defined prompt template
    )

    # Invoke the RetrievalQA with the provided query to get the result
    result = retrievalQA.invoke({"query": query})

    # Return only the generated answer from the result
    return result['result']


In [97]:
# Example usage
query = "Can you explain AI?"
#Generate an answer using the GPT-Neo model
result = generate_answer2(query,llm)
# Print the result
print(result)


Context: AI and Technology Knowledge BaseIntroduction to Artificial IntelligenceArtificial Intelligence (AI) is a branch of computer science that focuses on creating machinescapable of performing tasks that would normally require human intelligence. These tasksinclude learning, problem-solving, perception, language understanding, anddecision-making

. AI systems can be classified into two broad categories: narrow AI, which isdesigned to perform a narrow task (such as facial recognition or internet searches), andgeneral AI, which has the ability to perform any intellectual task that a human can do. AI istransforming industries, from healthcare and finance to transportation and manufacturing.The evolution of AI can be traced back to early philosophical discussions about the nature ofthe mind and mechanical reasoning

Question: Can you explain AI?

Answer: 

In the context of this question, you are probably asking a follow up question about the scope and meaning of AI. The answer may be 

In [178]:
# Example usage
query = "Can you explain AI?"
#Generate an answer using the GPT 2 model
result = generate_answer2(query,llm2)
# Print the result
print(result)


Context: AI and Technology Knowledge BaseIntroduction to Artificial IntelligenceArtificial Intelligence (AI) is a branch of computer science that focuses on creating machinescapable of performing tasks that would normally require human intelligence. These tasksinclude learning, problem-solving, perception, language understanding, anddecision-making

. AI systems can be classified into two broad categories: narrow AI, which isdesigned to perform a narrow task (such as facial recognition or internet searches), andgeneral AI, which has the ability to perform any intellectual task that a human can do. AI istransforming industries, from healthcare and finance to transportation and manufacturing.The evolution of AI can be traced back to early philosophical discussions about the nature ofthe mind and mechanical reasoning

Question: Can you explain AI?

Answer: 

. AI is the technology we use to do things. It is the type of technology that humans are supposed to use to do things.

. Artificia

In [174]:
# Example usage
query = "Can you explain AI?"
#Generate an answer using the GPT 2 model
result = generate_answer2(query,llm2)
# Print the result
print(result)


Context: AI and Technology Knowledge BaseIntroduction to Artificial IntelligenceArtificial Intelligence (AI) is a branch of computer science that focuses on creating machinescapable of performing tasks that would normally require human intelligence. These tasksinclude learning, problem-solving, perception, language understanding, anddecision-making

. AI systems can be classified into two broad categories: narrow AI, which isdesigned to perform a narrow task (such as facial recognition or internet searches), andgeneral AI, which has the ability to perform any intellectual task that a human can do. AI istransforming industries, from healthcare and finance to transportation and manufacturing.The evolution of AI can be traced back to early philosophical discussions about the nature ofthe mind and mechanical reasoning

Question: Can you explain AI?

Answer: 

This is an interesting question. First of all, it is important to distinguish the two, because they are fundamentally different. On